In [8]:
40 & 0x0fff

40

In [11]:
bin(0x0fff)

'0b111111111111'

In [16]:
print(bin(4097), '&', bin(0x0fff), '=', bin(4097&0x0fff))

0b1000000000001 & 0b111111111111 = 0b1


In [85]:
number = 5620042320032005
print(bin(number))
number &= 0xf000
bin(number >> 12)

0b10011111101110110011000010010000110111011000100000101


'0b1011'

In [80]:
len(bin(5620042320032005))

55

In [82]:
hex(0b1111111111111111100000000000000000000)

'0x1ffff00000'

In [67]:
hex(0b1111000000000000)

'0xf000'

In [89]:
BLOCK_SIZE = 8
import random
# проверка длины блока
assert not BLOCK_SIZE % 8, 'Длина блока должна быть кратна 8'

# вычисление контрольных бит
CHECK_BITS = [i for i in range(1, BLOCK_SIZE + 1) if not i & (i - 1)]


def chars_to_bin(chars):
    """
    Преобразование символов в бинарный формат
    """
    print(chars)
    assert not len(chars) * 8 % BLOCK_SIZE, 'Длина кодируемых данных должна быть кратна длине блока кодирования'
    return ''.join([bin(ord(c))[2:].zfill(8) for c in chars])


def block_iterator(text_bin, block_size=BLOCK_SIZE):
    """
    Поблочный вывод бинарных данных
    """
    for i in range(len(text_bin)):
        if not i % block_size:
            yield text_bin[i:i + block_size]


def get_check_bits_data(value_bin):
    """
    Получение информации о контрольных битах из бинарного блока данных
    """
    print('C1', len(value_bin), '\nC2 ', value_bin)
    check_bits_count_map = {k: 0 for k in CHECK_BITS}
    print('C3', CHECK_BITS)
    for index, value in enumerate(value_bin, 1):
        if int(value):
            bin_char_list = list(bin(index)[2:].zfill(8))
            bin_char_list.reverse()
            print('C4', bin_char_list)
            for degree in [2 ** int(i) for i, value in enumerate(bin_char_list) if int(value)]:
                print('C5', degree)
                print('C6', [2 ** int(i) for i, value in enumerate(bin_char_list) if int(value)])
                check_bits_count_map[degree] += 1
    check_bits_value_map = {}
    for check_bit, count in check_bits_count_map.items():
        check_bits_value_map[check_bit] = 0 if not count % 2 else 1
    return check_bits_value_map


def set_empty_check_bits(value_bin):
    """
    Добавить в бинарный блок "пустые" контрольные биты
    """
    for bit in CHECK_BITS:
        value_bin = value_bin[:bit - 1] + '0' + value_bin[bit - 1:]
    return value_bin


def set_check_bits(value_bin):
    """
    Установить значения контрольных бит
    """
    value_bin = set_empty_check_bits(value_bin)
    check_bits_data = get_check_bits_data(value_bin)
    for check_bit, bit_value in check_bits_data.items():
        value_bin = '{0}{1}{2}'.format(
            value_bin[:check_bit - 1], bit_value, value_bin[check_bit:])
    return value_bin


def get_check_bits(value_bin):
    """
    Получить информацию о контрольных битах из блока бинарных данных
    """
    check_bits = {}
    for index, value in enumerate(value_bin, 1):
        if index in CHECK_BITS:
            check_bits[index] = int(value)
    return check_bits


def exclude_check_bits(value_bin):
    """
    Исключить информацию о контрольных битах из блока бинарных данных
    """
    clean_value_bin = ''
    for index, char_bin in enumerate(list(value_bin), 1):
        if index not in CHECK_BITS:
            clean_value_bin += char_bin

    return clean_value_bin


def set_errors(encoded):
    """
    Допустить ошибку в блоках бинарных данных
    """
    result = ''
    for chunk in block_iterator(encoded, BLOCK_SIZE + len(CHECK_BITS)):
        num_bit = random.randint(1, len(chunk))
        chunk = '{0}{1}{2}'.format(chunk[:num_bit - 1], int(chunk[num_bit - 1]) ^ 1, chunk[num_bit:])
        result += (chunk)
    return result


def check_and_fix_error(encoded_chunk):
    """
    Проверка и исправление ошибки в блоке бинарных данных
    """
    check_bits_encoded = get_check_bits(encoded_chunk)
    check_item = exclude_check_bits(encoded_chunk)
    check_item = set_check_bits(check_item)
    check_bits = get_check_bits(check_item)
    if check_bits_encoded != check_bits:
        invalid_bits = []
        for check_bit_encoded, value in check_bits_encoded.items():
            if check_bits[check_bit_encoded] != value:
                invalid_bits.append(check_bit_encoded)
        num_bit = sum(invalid_bits)
        encoded_chunk = '{0}{1}{2}'.format(
            encoded_chunk[:num_bit - 1],
            int(encoded_chunk[num_bit - 1]) ^ 1,
            encoded_chunk[num_bit:])
    return encoded_chunk


def get_diff_index_list(value_bin1, value_bin2):
    """
    Получить список индексов различающихся битов
    """
    diff_index_list = []
    for index, char_bin_items in enumerate(zip(list(value_bin1), list(value_bin2)), 1):
        if char_bin_items[0] != char_bin_items[1]:
            diff_index_list.append(index)
    return diff_index_list


def encode(source):
    """
    Кодирование данных
    """
    text_bin = chars_to_bin(source)
    result = ''
    for chunk_bin in block_iterator(text_bin):
        chunk_bin = set_check_bits(chunk_bin)
        result += chunk_bin
    return result


def decode(encoded, fix_errors=True):
    """
    Декодирование данных
    """
    decoded_value = ''
    fixed_encoded_list = []
    for encoded_chunk in block_iterator(encoded, BLOCK_SIZE + len(CHECK_BITS)):
        if fix_errors:
            encoded_chunk = check_and_fix_error(encoded_chunk)
        fixed_encoded_list.append(encoded_chunk)

    clean_chunk_list = []
    for encoded_chunk in fixed_encoded_list:
        encoded_chunk = exclude_check_bits(encoded_chunk)
        clean_chunk_list.append(encoded_chunk)

    for clean_chunk in clean_chunk_list:
        for clean_char in [clean_chunk[i:i + 8] for i in range(len(clean_chunk)) if not i % 8]:
            decoded_value += chr(int(clean_char, 2))
    return decoded_value



source = input('Укажите текст для кодирования/декодирования:')
print('Длина блока кодирования: {0}'.format(BLOCK_SIZE))
print('Контрольные биты: {0}'.format(CHECK_BITS))
encoded = encode(source)
print('Закодированные данные: {0}'.format(encoded))
decoded = decode(encoded)
print('Результат декодирования: {0}'.format(decoded))
encoded_with_error = set_errors(encoded)
print('Допускаем ошибки в закодированных данных: {0}'.format(encoded_with_error))
diff_index_list = get_diff_index_list(encoded, encoded_with_error)
print('Допущены ошибки в битах: {0}'.format(diff_index_list))
decoded = decode(encoded_with_error, fix_errors=False)
print('Результат декодирования ошибочных данных без исправления ошибок: {0}'.format(decoded))
decoded = decode(encoded_with_error)
print('Результат декодирования ошибочных данных с исправлением ошибок: {0}'.format(decoded))

Длина блока кодирования: 8
Контрольные биты: [1, 2, 4, 8]
SEREGA NE MOLCHI BLIN
C1 12 
C2  000010100011
C3 [1, 2, 4, 8]
C4 ['1', '0', '1', '0', '0', '0', '0', '0']
C5 1
C6 [1, 4]
C5 4
C6 [1, 4]
C4 ['1', '1', '1', '0', '0', '0', '0', '0']
C5 1
C6 [1, 2, 4]
C5 2
C6 [1, 2, 4]
C5 4
C6 [1, 2, 4]
C4 ['1', '1', '0', '1', '0', '0', '0', '0']
C5 1
C6 [1, 2, 8]
C5 2
C6 [1, 2, 8]
C5 8
C6 [1, 2, 8]
C4 ['0', '0', '1', '1', '0', '0', '0', '0']
C5 4
C6 [4, 8]
C5 8
C6 [4, 8]
C1 12 
C2  000010000101
C3 [1, 2, 4, 8]
C4 ['1', '0', '1', '0', '0', '0', '0', '0']
C5 1
C6 [1, 4]
C5 4
C6 [1, 4]
C4 ['0', '1', '0', '1', '0', '0', '0', '0']
C5 2
C6 [2, 8]
C5 8
C6 [2, 8]
C4 ['0', '0', '1', '1', '0', '0', '0', '0']
C5 4
C6 [4, 8]
C5 8
C6 [4, 8]
C1 12 
C2  000010100010
C3 [1, 2, 4, 8]
C4 ['1', '0', '1', '0', '0', '0', '0', '0']
C5 1
C6 [1, 4]
C5 4
C6 [1, 4]
C4 ['1', '1', '1', '0', '0', '0', '0', '0']
C5 1
C6 [1, 2, 4]
C5 2
C6 [1, 2, 4]
C5 4
C6 [1, 2, 4]
C4 ['1', '1', '0', '1', '0', '0', '0', '0']
C5 1
C6 [1, 2, 8]


In [11]:
bytearray(encoded, 'utf8')

bytearray(b'010011011000100111000101000011001100000011001100110111001111010101000000100011110111110111001111110111110010000011001100100011010100010101000000010111001001010101000000100011010100110111001111110011011110010111110100010101000000100111011011110011011110110111001111100011110111010101000000010011011000110111001111100011110111010101000000010111110100110111001111010101000000000111000011110111110010100011101001')

In [9]:
def encoder(filein,fileout):
    blocksize = 8
    while True:
        s = filein.read(blocksize)
        if not s:
            break
        result = set_check_bits(chars_to_bin(s))
        fileout.write(result)

In [10]:
def decoder(filein, fileout, filelog):
    blocksize = 8
    s = filein.read(blocksize)
    if not s:
        return
    while True:
        fileout.write(s)
        s = filein.read(blocksize)
        if not s:
            break

In [8]:
bytearray('101', encoding=)

TypeError: string argument without an encoding

In [90]:
type("State".encode())

bytes

In [91]:
pos = 0
state = 0

In [92]:
print(f"Pos: {pos}" + f" State: {state}" + 
            f"time")

Pos: 0State: 0time


In [98]:
def chars_to_bin(chars):
    """
    Преобразование символов в бинарный формат
    """
    print(chars)
    return ''.join([bin(ord(c))[2:].zfill(8) for c in chars])


def block_iterator(text_bin, block_size: int):
    """
    Поблочный вывод бинарных данных
    """
    for i in range(len(text_bin)):
        if not i % block_size:
            yield text_bin[i:i + block_size]


def get_check_bits_data(value_bin: list, check_bits: list) -> map:
    """
    Получение информации о контрольных битах из бинарного блока данных
    """
    print('C1', len(value_bin), '\nC2 ', value_bin)
    check_bits_count_map = {k: 0 for k in check_bits}
    print('C3', check_bits)
    for index, value in enumerate(value_bin, 1):
        if int(value):
            bin_char_list = list(bin(index)[2:].zfill(8))
            bin_char_list.reverse()
            print('C4', bin_char_list)
            for degree in [2 ** int(i) for i, value in enumerate(bin_char_list) if int(value)]:
                print('C5 ', degree)
                print('C6 ', [2 ** int(i) for i, value in enumerate(bin_char_list) if int(value)])
                check_bits_count_map[degree] += 1
    check_bits_value_map = {}
    for check_bit, count in check_bits_count_map.items():
        check_bits_value_map[check_bit] = 0 if not count % 2 else 1
    return check_bits_value_map


def set_empty_check_bits(value_bin: list, check_bits: list) -> list:
    """
    Добавить в бинарный блок "пустые" контрольные биты
    """
    for bit in check_bits:
        value_bin = value_bin[:bit - 1] + '0' + value_bin[bit - 1:]
    return value_bin


def set_check_bits(value_bin: list, check_bits: list):
    """
    Установить значения контрольных бит
    """
    value_bin = set_empty_check_bits(value_bin, check_bits)
    check_bits_data = get_check_bits_data(value_bin, check_bits)
    for check_bit, bit_value in check_bits_data.items():
        value_bin = '{0}{1}{2}'.format(
            value_bin[:check_bit - 1], bit_value, value_bin[check_bit:])
    return value_bin


def get_check_bits(value_bin: list, check_bits: list) -> map:
    """
    Получить информацию о контрольных битах из блока бинарных данных
    """
    check_bits_map = {}
    for index, value in enumerate(value_bin, 1):
        if index in check_bits:
            check_bits_map[index] = int(value)
    return check_bits_map


def exclude_check_bits(value_bin: list, check_bits: list) -> str:
    """
    Исключить информацию о контрольных битах из блока бинарных данных
    """
    clean_value_bin = ''
    for index, char_bin in enumerate(list(value_bin), 1):
        if index not in check_bits:
            clean_value_bin += char_bin

    return clean_value_bin

def check_and_fix_error(encoded_chunk, check_bits):
    """
    Проверка и исправление ошибки в блоке бинарных данных
    """
    check_bits_encoded = get_check_bits(encoded_chunk, check_bits)
    check_item = exclude_check_bits(encoded_chunk, check_bits)
    check_item = set_check_bits(check_item, check_bits)
    check_bits_ = get_check_bits(check_item, check_bits)
    if check_bits_encoded != check_bits_:
        invalid_bits = []
        for check_bit_encoded, value in check_bits_encoded.items():
            if check_bits_[check_bit_encoded] != value:
                invalid_bits.append(check_bit_encoded)
        num_bit = sum(invalid_bits)
        encoded_chunk = '{0}{1}{2}'.format(
            encoded_chunk[:num_bit - 1],
            int(encoded_chunk[num_bit - 1]) ^ 1,
            encoded_chunk[num_bit:])
    return encoded_chunk


def get_diff_index_list(value_bin1, value_bin2):
    """
    Получить список индексов различающихся битов
    """
    diff_index_list = []
    for index, char_bin_items in enumerate(zip(list(value_bin1), list(value_bin2)), 1):
        if char_bin_items[0] != char_bin_items[1]:
            diff_index_list.append(index)
    return diff_index_list


In [99]:
def decoder(filein, fileout, filelog):
    blocksize = 9
    s = filein.read(blocksize)
    check_bits = [i for i in range(1, blocksize + 1) if not i & (i - 1)]
    if not s:
        return
    while True:
        decoded_value = ''
        fixed_encoded_list = []
        for encoded_chunk in block_iterator(s, blocksize + len(check_bits)):
            encoded_chunk = check_and_fix_error(encoded_chunk, check_bits)
            fixed_encoded_list.append(encoded_chunk)
        clean_chunk_list = []
        for encoded_chunk in fixed_encoded_list:
            encoded_chunk = exclude_check_bits(encoded_chunk, check_bits)
            clean_chunk_list.append(encoded_chunk)
        for clean_chunk in clean_chunk_list:
            for clean_char in [clean_chunk[i:i + 8] for i in range(len(clean_chunk)) if not i % 8]:
                decoded_value += chr(int(clean_char, 2))
        fileout.write(s)
        s = filein.read(blocksize)
        if not s:
            break


In [101]:
#!/usr/bin/python3
def encoder(filein,fileout):
    blocksize=8
    check_bits = [i for i in range(1, blocksize + 1) if not i & (i - 1)]
    print(check_bits)
    # while True:
    s=filein.read()
    # if not s:
    #     break
    result = chars_to_bin(s.decode('utf8'))
    print(len(result))
    result = set_check_bits(result, check_bits)
    fileout.write(result)

In [102]:
filein = open('fileIN_encoder.txt', 'rb')
fileout = open('fileOUT_encoder.txt', 'wb')
encoder(filein, fileout)

[1, 2, 4, 8]
SEREGA NE MOLCHI BLIN 
176
C1 180 
C2  000010100011010001010101001001000101010001110100000100100000010011100100010100100000010011010100111101001100010000110100100001001001001000000100001001001100010010010100111000100000
C3 [1, 2, 4, 8]
C4 ['1', '0', '1', '0', '0', '0', '0', '0']
C5  1
C6  [1, 4]
C5  4
C6  [1, 4]
C4 ['1', '1', '1', '0', '0', '0', '0', '0']
C5  1
C6  [1, 2, 4]
C5  2
C6  [1, 2, 4]
C5  4
C6  [1, 2, 4]
C4 ['1', '1', '0', '1', '0', '0', '0', '0']
C5  1
C6  [1, 2, 8]
C5  2
C6  [1, 2, 8]
C5  8
C6  [1, 2, 8]
C4 ['0', '0', '1', '1', '0', '0', '0', '0']
C5  4
C6  [4, 8]
C5  8
C6  [4, 8]
C4 ['0', '1', '1', '1', '0', '0', '0', '0']
C5  2
C6  [2, 4, 8]
C5  4
C6  [2, 4, 8]
C5  8
C6  [2, 4, 8]
C4 ['0', '1', '0', '0', '1', '0', '0', '0']
C5  2
C6  [2, 16]
C5  16
C6  [2, 16]


KeyError: 16